In [ ]:
pip install google-api-python-client pandas

  Using cached httplib2-0.31.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached uritemplate-4.2.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from googleapiclient.discovery import build
import pandas as pd

print("Environment ready")

Environment ready


In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
CHANNEL_ID = "UCq-Fj5jknLsUf-MWSy4_brA"
API_KEY = os.getenv("YOUTUBE_API_KEY")

In [2]:
from googleapiclient.discovery import build
import pandas as pd


In [22]:
CHANNEL_ID = "UCq-Fj5jknLsUf-MWSy4_brA" 

In [23]:
youtube = build(
    serviceName="youtube",
    version="v3",
    developerKey=API_KEY
)

print("YouTube API connected successfully")


YouTube API connected successfully


In [24]:
channel_request = youtube.channels().list(
    part="snippet,statistics",
    id=CHANNEL_ID
)
channel_response = channel_request.execute()
channel_info = channel_response["items"][0]
channel_data = {
    "channel_id": CHANNEL_ID,
    "channel_name": channel_info["snippet"]["title"],
    "subscribers": int(channel_info["statistics"].get("subscriberCount", 0)),
    "total_videos": int(channel_info["statistics"].get("videoCount", 0)),
    "total_views": int(channel_info["statistics"].get("viewCount", 0))
}

channel_df = pd.DataFrame([channel_data])
channel_df


,channel_id,channel_name,subscribers,total_videos,total_views
0,UCq-Fj5jknLsUf-MWSy4_brA,T-Series,308000000,24985,323166782568


In [25]:
video_ids = []

search_request = youtube.search().list(
    part="id",
    channelId=CHANNEL_ID,
    maxResults=80,
    order="date",
    type="video"
)

search_response = search_request.execute()

for item in search_response["items"]:
    video_ids.append(item["id"]["videoId"])

len(video_ids)


50

In [26]:
videos_data = []

video_request = youtube.videos().list(
    part="snippet,statistics",
    id=",".join(video_ids)
)

video_response = video_request.execute()

for video in video_response["items"]:
    video_data = {
        "video_id": video["id"],
        "channel_id": CHANNEL_ID,
        "title": video["snippet"]["title"],
        "published_date": video["snippet"]["publishedAt"],
        "views": int(video["statistics"].get("viewCount", 0)),
        "likes": int(video["statistics"].get("likeCount", 0)),
        "comments": int(video["statistics"].get("commentCount", 0))
    }
    videos_data.append(video_data)

videos_df = pd.DataFrame(videos_data)
videos_df.head()


,video_id,channel_id,title,published_date,views,likes,comments
0,OBLQTYgCaDo,UCq-Fj5jknLsUf-MWSy4_brA,Maa Ka Darbaar (Audio): Sonu Nigam | Shabbir A...,2025-12-15T11:28:12Z,120,24,15
1,HtwMp77a2Yk,UCq-Fj5jknLsUf-MWSy4_brA,Sawaari (Lyrical Video): Faridkot | IP Singh |...,2025-12-15T09:05:03Z,1194,60,36
2,HpSR9RSohUg,UCq-Fj5jknLsUf-MWSy4_brA,Listen to this romatic song #HaiyeRe,2025-12-15T08:34:37Z,11764,161,5
3,DSj58CoUNnE,UCq-Fj5jknLsUf-MWSy4_brA,Single Papa: Ib Na Jaane (Song) | Kunal Kemmu ...,2025-12-15T05:30:05Z,117162,997,78
4,zSmBw5xIDjM,UCq-Fj5jknLsUf-MWSy4_brA,Sahana Sahana Promo (Hindi) - The RajaSaab | P...,2025-12-14T13:21:41Z,322064,19441,825


In [27]:
channel_df.to_csv("../data/raw/channel_raw.csv", index=False)
videos_df.to_csv("../data/raw/videos_raw.csv", index=False)

print("Week-1 raw data saved successfully")


Week-1 raw data saved successfully
